In [1]:
import numpy as np
import matplotlib.pyplot as plt
from utils.notebook_config import *
import random
from utils.simulation import *

In [11]:
road = rna(L=30, tau=1, steps=10)

ribo1 = ribo(rna=road)

ribo1.move(rna=road)
ribo1.move(rna=road)
ribo1.move(rna=road)

ribo2 = ribo(rna=road)

ribo2.move(rna=road)
ribo2.move(rna=road)
ribo2.move(rna=road)
ribo2.move(rna=road)
ribo2.move(rna=road)

print(ribo2.position())
print(road.occupation())

2
[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
